In [5]:
import sys
sys.path.insert(0, r"C:\Users\PC\Desktop\graphglue")
from graphglue.core.graph import Graph


In [7]:
# ---------- Setup ----------
G = Graph(directed=True)
conditions = ["Healthy", "Stressed", "Disease"]
for c in conditions:
    G.add_layer(c, condition=c)

# Entities
proteins = [f"P{i}" for i in range(1, 151)]   # P1..P150
transcripts = [f"T{i}" for i in range(1, 61)]  # T1..T60 (treat as vertices)
enz_edge_entities = [f"edge_rxn_{i}" for i in range(1, 11)]  # edge-entities for reactions

# Seed some vertex attributes
for p in proteins[:10]:
    G.add_vertex(p, layer="Healthy", family="kinase")
for p in proteins[10:]:
    G.add_vertex(p, layer="Healthy")
for t in transcripts:
    G.add_vertex(t, layer="Healthy", kind="transcript")
for ee in enz_edge_entities:
    G.add_edge_entity(ee, layer="Healthy", role="enzyme")

# Propagate initial vertices to all layers (cheaply)
for lid in ["Stressed", "Disease"]:
    G._layers[lid]["vertices"].update(G._layers["Healthy"]["vertices"])

In [9]:
# ---------- Build PPI edges in all layers ----------
import random

def rand_weight(base=1.0, jitter=0.5):
    return max(0.05, base + (random.random() - 0.5) * 2 * jitter)

ppis = []
for _ in range(320):
    u, v = random.sample(proteins, 2)
    w = rand_weight(1.2, 0.6)
    e = G.add_edge(u, v, layer="Healthy", weight=w, edge_directed=False)
    ppis.append(e)

# Stress/disease layer variants (override per-layer weights)
for eid in ppis:
    # Stressed: mostly +10% with jitter
    G.add_edge_to_layer("Stressed", eid)
    G.set_edge_layer_attrs("Stressed", eid, weight=G.edge_weights[eid] * rand_weight(1.10, 0.1))
    # Disease: some edges get weaker; others stronger
    G.add_edge_to_layer("Disease", eid)
    factor = 0.7 if random.random() < 0.4 else 1.3
    G.set_edge_layer_attrs("Disease", eid, weight=G.edge_weights[eid] * rand_weight(factor, 0.15))

In [13]:
# ---------- Complexes as undirected hyperedges ----------
complexes = []
for _ in range(12):
    members = set(random.sample(proteins, random.choice([3, 4, 5])))
    hid = G.add_hyperedge(members=members, layer="Healthy", weight=rand_weight(1.0, 0.2))
    complexes.append(hid)
    # complex exists in all layers (same membership)
    for lid in ["Stressed", "Disease"]:
        G.add_edge_to_layer(lid, hid)

In [15]:
# ---------- Directed signaling cascades as hyperedges ----------
cascades = []
while len(cascades) < 8:
    head = set(random.sample(proteins, random.choice([1, 2])))
    tail = set(random.sample(proteins, random.choice([2, 3, 4])))
    if head & tail:
        continue  # resample until disjoint
    hid = G.add_hyperedge(head=head, tail=tail, layer="Healthy", weight=rand_weight(1.0, 0.4))
    cascades.append(hid)
    for lid in ["Stressed", "Disease"]:
        G.add_edge_to_layer(lid, hid)

In [17]:
# ---------- Reactions connecting vertices to edge-entities ----------
for ee in enz_edge_entities:
    s, t = random.sample(proteins, 2)
    G.add_edge(s, ee, layer="Healthy", edge_type="vertex_edge", weight=1.0 + random.random())
    G.add_edge(ee, t, layer="Healthy", edge_type="vertex_edge", weight=1.0 + random.random())
    # propagate across layers
    for lid in ["Stressed", "Disease"]:
        G._layers[lid]["edges"].update(G._layers["Healthy"]["edges"])

In [19]:
# ---------- Basic sanity ----------
print("vertices:", G.number_of_vertices(), "Edges:", G.number_of_edges())

# Only true "vertices" are counted by number_of_vertices() (proteins + transcripts)
expected_vertices = len(set(proteins)) + len(set(transcripts))  # 150 + 60 = 210
assert G.number_of_vertices() >= expected_vertices, f"Expected ≥{expected_vertices}, got {G.number_of_vertices()}"

# Edge-entities are tracked as entity_type == 'edge' (not included in number_of_vertices)
edge_entity_ids = set(enz_edge_entities)
edge_entity_count = sum(1 for _id, et in G.entity_types.items() if et == "edge" and _id in edge_entity_ids)
assert edge_entity_count == len(edge_entity_ids), f"Expected {len(edge_entity_ids)} edge-entities, got {edge_entity_count}"

# Edges: PPIs (320) + complexes (12) + cascades (8) + reaction links (10*2) = 360 minimum
assert G.number_of_edges() >= 320 + 12 + 8 + (10 * 2)


vertices: 210 Edges: 360


In [21]:
# ---------- Views & top edges by condition ----------
import polars as pl

for cond in conditions:
    EV = G.edges_view(layer=cond, resolved_weight=True)
    print(f"[{cond}] edges_view rows =", EV.height)
    top = (
        EV
        .filter(pl.col("kind") == "binary")
        .sort("effective_weight", descending=True)
        .select(["edge_id", "source", "target", "effective_weight"])
        .head(5)
    )
    print(f"\nTop 5 binary edges by effective_weight in {cond}:")
    print(top)

[Healthy] edges_view rows = 360

Top 5 binary edges by effective_weight in Healthy:
shape: (5, 4)
┌──────────┬────────┬────────────┬──────────────────┐
│ edge_id  ┆ source ┆ target     ┆ effective_weight │
│ ---      ┆ ---    ┆ ---        ┆ ---              │
│ str      ┆ str    ┆ str        ┆ f64              │
╞══════════╪════════╪════════════╪══════════════════╡
│ edge_354 ┆ P88    ┆ edge_rxn_8 ┆ 1.825315         │
│ edge_191 ┆ P34    ┆ P129       ┆ 1.796666         │
│ edge_222 ┆ P134   ┆ P111       ┆ 1.79373          │
│ edge_131 ┆ P11    ┆ P31        ┆ 1.792635         │
│ edge_88  ┆ P128   ┆ P79        ┆ 1.792061         │
└──────────┴────────┴────────────┴──────────────────┘
[Stressed] edges_view rows = 360

Top 5 binary edges by effective_weight in Stressed:
shape: (5, 4)
┌──────────┬────────┬────────┬──────────────────┐
│ edge_id  ┆ source ┆ target ┆ effective_weight │
│ ---      ┆ ---    ┆ ---    ┆ ---              │
│ str      ┆ str    ┆ str    ┆ f64              │
╞═══════

In [23]:
# ---------- Layer analytics ----------
stats = G.layer_statistics()
print("\nLayer stats:", stats)

conserved = G.conserved_edges(min_layers=3)  # present in all 3 conditions
print("\nConserved edges (in all conditions):", len(conserved))

disease_specific = G.layer_specific_edges("Disease")
print("Disease-specific edges:", len(disease_specific))

changes = G.temporal_dynamics(["Healthy", "Stressed", "Disease"], metric="edge_change")
print("\nTemporal edge changes (Healthy→Stressed→Disease):", changes)
assert len(changes) == 2


Layer stats: {'Healthy': {'vertices': 220, 'edges': 360, 'attributes': {'condition': 'Healthy'}}, 'Stressed': {'vertices': 220, 'edges': 360, 'attributes': {'condition': 'Stressed'}}, 'Disease': {'vertices': 220, 'edges': 360, 'attributes': {'condition': 'Disease'}}}

Conserved edges (in all conditions): 360
Disease-specific edges: 0

Temporal edge changes (Healthy→Stressed→Disease): [{'added': 0, 'removed': 0, 'net_change': 0}, {'added': 0, 'removed': 0, 'net_change': 0}]


In [25]:
# ---------- Presence queries ----------
some_e = next(iter(G.edge_to_idx.keys()))
print("\nEdge presence for", some_e, ":", G.edge_presence_across_layers(edge_id=some_e))
some_p = random.choice(proteins)
print("vertex presence for", some_p, ":", G.vertex_presence_across_layers(some_p))


Edge presence for edge_0 : ['Healthy', 'Stressed', 'Disease']
vertex presence for P80 : ['Healthy', 'Stressed', 'Disease']


In [27]:
# ---------- Traversal checks ----------
q = random.choice(proteins)
print(f"\nNeighbors({q}) =>", G.neighbors(q)[:10])
print(f"Out({q}) =>", G.out_neighbors(q)[:10])
print(f"In({q}) =>", G.in_neighbors(q)[:10])


Neighbors(P37) => ['P64', 'P130', 'P52']
Out(P37) => ['P64', 'P130', 'P52']
In(P37) => ['P64', 'P130', 'P52']


In [29]:
# ---------- Subgraph slice & copy ----------
H = G.subgraph_from_layer("Disease", resolve_layer_weights=True)
assert set(H.vertices()).issubset(set(G.vertices()))
assert set(H.edges()).issubset(set(G.edges()))
print("\nDisease subgraph: vertices =", H.number_of_vertices(), "edges =", H.number_of_edges())

K = G.copy()
assert set(K.vertices()) == set(G.vertices())
assert set(K.edges()) == set(G.edges())
# hyperedge shape preserved
any_hyper = next(e for e,k in G.edge_kind.items() if k == "hyper")
assert K.edge_kind.get(any_hyper) == "hyper"
# layer sets preserved
for lid in G.list_layers(include_default=True):
    assert K._layers[lid]["vertices"] == G._layers[lid]["vertices"]
    assert K._layers[lid]["edges"] == G._layers[lid]["edges"]
print("copy() OK")


Disease subgraph: vertices = 210 edges = 360
copy() OK


In [31]:
# ---------- Remove operations stress ----------
to_drop_vertices = random.sample(proteins, 5)
for n in to_drop_vertices:
    if n in G.entity_to_idx:
        G.remove_vertex(n)
print("\nAfter removing 5 proteins: vertices =", G.number_of_vertices(), "edges =", G.number_of_edges())

to_drop_edges = list(G.edge_to_idx.keys())[:10]
for eid in to_drop_edges:
    if eid in G.edge_to_idx:
        G.remove_edge(eid)
print("After removing 10 edges: vertices =", G.number_of_vertices(), "edges =", G.number_of_edges())


After removing 5 proteins: vertices = 205 edges = 342
After removing 10 edges: vertices = 205 edges = 332


In [33]:
# ---------- Audit & memory ----------
audit = G.audit_attributes()
print("\nAudit:", audit)
mem_bytes = G.memory_usage()
print("Approx memory usage (bytes):", int(mem_bytes))
assert mem_bytes > 0

print("\nReality-check finished ✅")


Audit: {'extra_vertex_rows': ['edge_rxn_10', 'edge_rxn_8', 'edge_rxn_5', 'edge_rxn_4', 'edge_rxn_7', 'edge_rxn_3', 'edge_rxn_2', 'edge_rxn_9', 'edge_rxn_6', 'edge_rxn_1'], 'extra_edge_rows': [], 'missing_vertex_rows': [], 'missing_edge_rows': ['edge_97', 'edge_28', 'edge_262', 'edge_251', 'edge_57', 'edge_228', 'edge_61', 'edge_183', 'edge_68', 'edge_335', 'edge_284', 'edge_50', 'edge_14', 'edge_326', 'edge_88', 'edge_299', 'edge_215', 'edge_83', 'edge_237', 'edge_312', 'edge_342', 'edge_105', 'edge_204', 'edge_90', 'edge_133', 'edge_313', 'edge_59', 'edge_45', 'edge_77', 'edge_161', 'edge_354', 'edge_176', 'edge_300', 'edge_243', 'edge_62', 'edge_201', 'edge_166', 'edge_43', 'edge_192', 'edge_297', 'edge_121', 'edge_203', 'edge_330', 'edge_122', 'edge_96', 'edge_146', 'edge_18', 'edge_48', 'edge_35', 'edge_73', 'edge_305', 'edge_171', 'edge_309', 'edge_229', 'edge_259', 'edge_188', 'edge_339', 'edge_125', 'edge_253', 'edge_112', 'edge_115', 'edge_117', 'edge_189', 'edge_22', 'edge_31

In [35]:
events = G.history()           # list[dict]
df = G.history(as_df=True)     # Polars DF [DataFrame]

In [37]:
print(df.head())
events[:3]

shape: (5, 10)
┌─────────┬─────────────────────────────┬──────────┬─────────────────┬───┬────────┬───────────┬─────────┬────────────┐
│ version ┆ ts_utc                      ┆ mono_ns  ┆ op              ┆ … ┆ result ┆ vertex_id ┆ layer   ┆ attributes │
│ ---     ┆ ---                         ┆ ---      ┆ ---             ┆   ┆ ---    ┆ ---       ┆ ---     ┆ ---        │
│ i64     ┆ str                         ┆ i64      ┆ str             ┆   ┆ str    ┆ str       ┆ str     ┆ struct[1]  │
╞═════════╪═════════════════════════════╪══════════╪═════════════════╪═══╪════════╪═══════════╪═════════╪════════════╡
│ 1       ┆ 2025-10-13T10:40:32.078250Z ┆ 3932100  ┆ set_layer_attrs ┆ … ┆ null   ┆ null      ┆ null    ┆ null       │
│ 2       ┆ 2025-10-13T10:40:32.079235Z ┆ 4346900  ┆ set_layer_attrs ┆ … ┆ null   ┆ null      ┆ null    ┆ null       │
│ 3       ┆ 2025-10-13T10:40:32.079235Z ┆ 4770000  ┆ set_layer_attrs ┆ … ┆ null   ┆ null      ┆ null    ┆ null       │
│ 4       ┆ 2025-10-13T10:40:32.0

[{'version': 1,
  'ts_utc': '2025-10-13T10:40:32.078250Z',
  'mono_ns': 3932100,
  'op': 'set_layer_attrs',
  'layer_id': 'Healthy',
  'attrs': {'condition': 'Healthy'},
  'result': None},
 {'version': 2,
  'ts_utc': '2025-10-13T10:40:32.079235Z',
  'mono_ns': 4346900,
  'op': 'set_layer_attrs',
  'layer_id': 'Stressed',
  'attrs': {'condition': 'Stressed'},
  'result': None},
 {'version': 3,
  'ts_utc': '2025-10-13T10:40:32.079235Z',
  'mono_ns': 4770000,
  'op': 'set_layer_attrs',
  'layer_id': 'Disease',
  'attrs': {'condition': 'Disease'},
  'result': None}]

In [39]:
import sys, pathlib

# add repo root to Python path
sys.path.append(str(pathlib.Path.cwd().parent))
from graphglue.io import csv as graph_csv

In [41]:
csv1_path = "csv1_edges.csv"
pl.DataFrame({
    "source": ["A","A","B","C","D"],
    "target": ["B","C","C","D","A"],
    "weight": [1, 2, 3, 1, 5],
    "directed": [True, True, False, True, True],
    "layer": ["L1","L1","L1","L2","L2"],
}) #.write_csv(csv1_path)

#pl.read_csv(csv1_path).head()


source,target,weight,directed,layer
str,str,i64,bool,str
"""A""","""B""",1,true,"""L1"""
"""A""","""C""",2,true,"""L1"""
"""B""","""C""",3,false,"""L1"""
"""C""","""D""",1,true,"""L2"""
"""D""","""A""",5,true,"""L2"""


In [43]:
G = graph_csv.load_csv_to_graph(
    csv1_path,
    schema="auto",            # or 'edge_list'/'incidence'/'adjacency'/'hyperedge'/'lil'
    default_layer=None,       # fallback if no layer column is present
    default_directed=None,    # fallback if no directed column and cannot infer
    default_weight=1.0,
)

# Quick sanity: show first rows of an edges view (columns depend on your Graph implementation)
edges = G.edges_view(layer=None, include_directed=True, resolved_weight=True)
edges.head()


FileNotFoundError: Le fichier spécifié est introuvable. (os error 2): csv1_edges.csv

In [45]:
# Count entities and edges (attribute names based on your class; adjust if different)
num_entities = G.global_entity_count        # vertices + edge-entities
num_edges    = G.global_edge_count          # binary + hyper

print("entities:", num_entities, "edges:", num_edges)

# A light “degree” summary from edges_view for binary edges only (skip hyper)
df = G.edges_view(include_directed=True, resolved_weight=True)

# Try to find endpoint column names robustly
cols = {c.lower(): c for c in df.columns}
# Common possibilities:
src_col = next((cols[c] for c in ["source","src","u","from"]), None)
dst_col = next((cols[c] for c in ["target","dst","v","to"]), None)

if src_col and dst_col:
    # out-degree (directed) / degree (undirected)
    out_deg = df.group_by(src_col).len().rename({src_col: "vertex", "len": "out_degree"})
    in_deg  = df.group_by(dst_col).len().rename({dst_col: "vertex", "len": "in_degree"})
    deg = out_deg.join(in_deg, on="vertex", how="outer").fill_null(0)
    deg = deg.with_columns((pl.col("out_degree")+pl.col("in_degree")).alias("total_degree"))
    deg.sort("total_degree", descending=True).head(10)
else:
    print("Skip degree summary: endpoint columns not found in edges_view output (likely hyperedge-only or different schema).")


entities: <bound method Graph.global_entity_count of <graphglue.core.graph.Graph object at 0x000001679542E660>> edges: <bound method Graph.global_edge_count of <graphglue.core.graph.Graph object at 0x000001679542E660>>


C:\Users\pc\AppData\Local\Temp\ipykernel_27668\2468729116.py:20: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  deg = out_deg.join(in_deg, on="vertex", how="outer").fill_null(0)


In [47]:
# Add a new vertex and an edge on layer L3
G.add_vertex("E")
eid = G.add_edge("E", "A", layer="L3", directed=True, weight=2.5)

# Per-layer weight override example:
G.set_edge_layer_attrs("L3", eid, weight=3.0)

# Inspect the updated edges
G.edges_view(include_directed=True, resolved_weight=True).tail()


edge_id,kind,directed,global_weight,effective_weight,source,target,edge_type
str,str,bool,f64,f64,str,str,str
"""edge_355""","""binary""",true,1.501178,1.501178,"""edge_rxn_8""","""P6""","""vertex_edge"""
"""edge_356""","""binary""",true,1.357721,1.357721,"""P145""","""edge_rxn_9""","""vertex_edge"""
"""edge_357""","""binary""",true,1.246218,1.246218,"""edge_rxn_9""","""P90""","""vertex_edge"""
"""edge_358""","""binary""",true,1.504452,1.504452,"""P19""","""edge_rxn_10""","""vertex_edge"""
"""edge_360""","""binary""",true,2.5,2.5,"""E""","""A""","""regular"""


In [49]:
csv2_path = "csv2_edges_view.csv"
G.edges_view(layer=None, include_directed=True, resolved_weight=True)
csv2_path


'csv2_edges_view.csv'

In [51]:
def export_edge_list_csv(G, path, layer=None):
    df = G.edges_view(layer=layer, include_directed=True, resolved_weight=True)
    cols = {c.lower(): c for c in df.columns}
    src_col = next((cols[c] for c in ["source","src","u","from"]), None)
    dst_col = next((cols[c] for c in ["target","dst","v","to"]), None)
    dir_col = next((cols[c] for c in ["directed"]), None)
    w_eff   = next((cols[c] for c in ["effective_weight","weight","w"]), None)

    if not (src_col and dst_col):
        raise ValueError("No binary endpoint columns found; the view may be hyperedge-only. Try the generic edges_view export.")

    out = pl.DataFrame({
        "source": df[src_col],
        "target": df[dst_col],
        "weight": df[w_eff] if w_eff else pl.Series([1.0]*df.height),
        "directed": df[dir_col] if dir_col in df.columns else pl.Series([None]*df.height),
        "layer": pl.Series([layer]*df.height) if layer else pl.Series([None]*df.height),
    })
    out #.write_csv(path)

# Usage:
csv2_edge_list_path = "csv2_edge_list.csv"
export_edge_list_csv(G, csv2_edge_list_path, layer=None)
csv2_edge_list_path


'csv2_edge_list.csv'

In [53]:
# In-memory look at last few events
hist = G.history(as_df=True)   # DF [DataFrame]
hist.tail()

# Save to Parquet/CSV/JSON [JavaScript Object Notation]/NDJSON [Newline-Delimited JSON]
G.export_history("graph_history.parquet")


1271

In [55]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))

from graphglue.io import excel as graph_excel
from graphglue.core.graph import Graph


In [57]:
G = graph_excel.load_excel_to_graph("graph_input.xlsx", schema="auto")

In [59]:
G.global_entity_count()

60

In [61]:
from graphglue.adapters.networkx import to_backend, to_nx, from_nx

In [62]:
nxG, man = to_nx(G, directed=True, hyperedge_mode="skip")

In [64]:
from graphglue.adapters.igraph import to_backend, to_igraph, from_igraph

In [67]:
nxG, man = to_igraph(G, directed=True, hyperedge_mode="skip")

In [69]:
G.shape

(60, 767)

In [71]:
parts = G.nx.louvain_communities(G)                          # warns on loss if neededparts = G.nx.louvain_communities(G, _nx_directed=False)      # force undirected NX backendparts = G.nx.louvain_communities(G, _nx_hyperedge="expand")  # expand hyperedges

In [75]:
path = G.nx.shortest_path(G, source="B701", target="MS6 8")
dist = G.nx.shortest_path_length(G, source="B701", target="MS6 8")

In [79]:
print(dist)
path

2


['B701', 'MS8 21', 'MS6 8']

In [81]:
G.nx.clear()   

In [83]:
# predictable_proxy_tests.py
# Deterministic smoke tests for the lazy NX (NetworkX) proxy

from graphglue.core.graph import Graph

# ---------- G1: PATH GRAPH (for weighted/unweighted shortest paths) ----------
def build_path_graph() -> Graph:
    """
    Directed chain a→b→c→d→e→f with weights on each edge.
    - Weighted shortest path a→f = 1+2+3+1+4 = 11
    - Unweighted hops a→f = 5
    """
    G = Graph(directed=True)
    # vertices (+ labels for the label→ID mapping)
    G.add_vertex("a", name="alpha")
    G.add_vertex("b", name="bravo")
    G.add_vertex("c", name="charlie")
    G.add_vertex("d", name="delta")
    G.add_vertex("e", name="echo")
    G.add_vertex("f", name="phi")

    # pure chain (NO chords)
    G.add_edge("a", "b", weight=1)
    G.add_edge("b", "c", weight=2)
    G.add_edge("c", "d", weight=3)
    G.add_edge("d", "e", weight=1)
    G.add_edge("e", "f", weight=4)

    return G


# ---------- G2: COMMUNITY GRAPH (two cliques + weak bridge) ----------
def build_community_graph() -> Graph:
    """
    Two undirected cliques: K6 on {a..f} and K4 on {w,x,y,z}, joined by a single weak bridge e--x (weight=0.01).
    Louvain should give communities of sizes [4,6] (stable with seed); betweenness top in {'e','x'};
    PR (PageRank) top is 'e' in undirected view (highest degree).
    """
    G = Graph(directed=True)  # we’ll add undirected edges explicitly

    for v in ["a","b","c","d","e","f","w","x","y","z"]:
        G.add_vertex(v)

    # K6 clique on a..f (undirected, weight=1)
    k6 = ["a","b","c","d","e","f"]
    for i in range(len(k6)):
        for j in range(i+1, len(k6)):
            G.add_edge(k6[i], k6[j], weight=1, edge_directed=False)

    # K4 clique on w,x,y,z (undirected, weight=1)
    k4 = ["w","x","y","z"]
    for i in range(len(k4)):
        for j in range(i+1, len(k4)):
            G.add_edge(k4[i], k4[j], weight=1, edge_directed=False)

    # Single weak bridge e--x
    G.add_edge("e", "x", weight=0.01, edge_directed=False)

    return G

def run_tests():
    # ----- G1: shortest paths -----
    G1 = build_path_graph()

    # Weighted Dijkstra via labels -> expect 11.0
    dist_w = G1.nx.shortest_path_length(
        G1, source="alpha", target="phi", weight="weight", _nx_label_field="name"
    )
    print("[G1:dijkstra weighted] alpha->phi:", dist_w, "(expect 11.0)")

    # Unweighted hop count -> expect 5 (a-b-c-d-e-f)
    dist_hops = G1.nx.shortest_path_length(G1, source="a", target="f", weight=None)
    print("[G1:unweighted hops] a->f:", dist_hops, "(expect 5)")

    # Cache invalidation: add fast a->f edge (weight=2) and re-check -> expect 2.0
    G1.add_edge("a", "f", weight=2)
    dist_new = G1.nx.shortest_path_length(G1, source="a", target="f", weight="weight")
    print("[G1:after mutation] a->f:", dist_new, "(expect 2.0)")

    # ----- G2: communities / centrality / PR / components -----
    G2 = build_community_graph()

    # Louvain on undirected view -> expect sizes [4, 6]
    comms = G2.nx.louvain_communities(G2, _nx_directed=False, seed=42, weight="weight")
    sizes = sorted(len(c) for c in comms)
    print("[G2:louvain] sizes:", sizes, "(expect [4, 6])")

    # Betweenness centrality (unweighted, undirected) -> bridge endpoints dominate
    bc = G2.nx.betweenness_centrality(G2, _nx_directed=False, normalized=True)
    top_bc = max(bc, key=bc.get)
    print("[G2:betweenness] top:", top_bc, "(expect 'e' or 'x')")

    # PageRank (PR) on undirected view (unweighted edges) -> highest degree node is 'e'
    pr = G2.nx.pagerank(G2, _nx_directed=False)
    top_pr = max(pr, key=pr.get)
    print("[G2:pagerank] top:", top_pr, "(expect 'e')")

    # Connected components (undirected) -> single component of size 10
    comps = list(G2.nx.connected_components(G2, _nx_directed=False))
    print("[G2:connected components]:", [sorted(c) for c in comps], "(expect one component of size 10)")


if __name__ == "__main__":
    run_tests()


[G1:dijkstra weighted] alpha->phi: 11 (expect 11.0)
[G1:unweighted hops] a->f: 5 (expect 5)
[G1:after mutation] a->f: 2 (expect 2.0)
[G2:louvain] sizes: [4, 6] (expect [4, 6])
[G2:betweenness] top: e (expect 'e' or 'x')
[G2:pagerank] top: e (expect 'e')
[G2:connected components]: [['a', 'b', 'c', 'd', 'e', 'f', 'w', 'x', 'y', 'z']] (expect one component of size 10)
